<a href="https://colab.research.google.com/github/rabimba/GDE-ML-Artifacts/blob/main/Gemini_Video_Explain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
# Fetch the discovery docs for the Generative Language API service.
from googleapiclient.discovery import build
import googleapiclient
import requests

DISCOVERY_URL = f'https://generativelanguage.googleapis.com/$discovery/rest?version=v1beta&key={GOOGLE_API_KEY}';
discovery_docs = requests.get(DISCOVERY_URL).content
genai_service = googleapiclient.discovery.build_from_document(discovery_docs, developerKey=GOOGLE_API_KEY)

In [ ]:
import cv2
import os
import shutil

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = int(vidcap.get(cv2.CAP_PROP_FPS))
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  success,image = vidcap.read()
  frame_count = 0  # Initialize a frame counter
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success:  # End of video
          break
      if count % int(fps) == 0:  # Extract a frame every second
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{frame_count:04d}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release()  # Release the capture object
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")


video_file_name = "cs25.mp4" # @param {type:"string"}
extract_frame_from_video(video_file_name)

In [ ]:
import mimetypes
import os

class File:
  def __init__(self, file_path: str, display_name: str = None,
               timestamp_seconds: int = None, mimetype: str = None, uri = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    if timestamp_seconds != None:
      self.timestamp = seconds_to_time_string(timestamp_seconds)
    # Detect mimetype if not specified
    self.mimetype = mimetype if mimetype else mimetypes.guess_type(file_path)[0]
    self.uri = uri

  def set_file_uri(self, uri):
    self.uri = uri

def seconds_to_time_string(seconds):
  """Converts an integer number of seconds to a string in the format '00:00'.
     Format is the expected format for Gemini 1.5.
  """
  minutes = seconds // 60
  seconds = seconds % 60
  return f"{minutes:02d}:{seconds:02d}"

def get_timestamp_seconds(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame0000.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicate that the filename might be incorrectly formatted

  frame_count_str = parts[1].split(".")[0]
  return int(frame_count_str)

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)  # Sort alphabetically
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file),
           timestamp_seconds=get_timestamp_seconds(file)))

# Upload the files to the API
uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')
for file in files_to_upload:
  print(f'Uploading: {file.file_path}...')
  response = genai_service.media().upload(
      media_body=file.file_path,
      media_mime_type = file.mimetype).execute()
  file.set_file_uri(response["file"]["uri"])
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

In [ ]:
# List files uploaded in the API
response = genai_service.files().list().execute()
print(response)

In [ ]:
# Make GenerateContent Request
def makeGenerateContentRequest(prompt, files):
  generateContent = {"contents": [{ "parts": [{ "text": prompt }]}]}
  for file in files:
    generateContent["contents"][0]["parts"].extend(makeVideoPart(file))
  return generateContent

def makeVideoPart(file):
  return [
      {"text": file.timestamp},
      {"file_data": {"file_uri": file.uri, "mime_type": file.mimetype }}]

prompt = "Explain the video in detail. List the concepts of each slide and write code examples for each concept" # @param {type:"string"}
model = "models/gemini-1.5-pro-latest" # @param ["models/gemini-1.5-pro-latest", "models/gemini-1.0-pro-vision-latest"]
response = genai_service.models().generateContent(
    model = model,
    body = makeGenerateContentRequest(prompt, uploaded_files)).execute()
print(response)

## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.